In [1]:
import numpy as np
from scipy import signal


data1 = np.load("./datasets/Gabi.npy")
data2 = np.load("./datasets/Jesse.npy")

data1.shape, data2.shape

((28, 1000, 4), (28, 1000, 4))

In [2]:
# Correção número de eletrodos e unificação

# os dois primeiros eletrodos são funcionais
data1 = data1[:,:,0:2]
data2 = data2[:,:,0:2]

data = np.array([data1, data2])

print(data.shape)

# colocando os dados na dimensão final
data = data.transpose(0, 1, 3, 2)
data.shape

(2, 28, 1000, 2)


(2, 28, 2, 1000)

In [3]:
# Funções para aplicação dos filtros temporais

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

*Tarefa 1*: Realizem a aplicaçao dos filtros para toda a base.


In [4]:
# Aplicação dos filtros


# Mostrar até  (5 - 50hz)


In [5]:
# juntando participantes (apenas para demonstração)
data = data.reshape(data.shape[0] * data.shape[1], data.shape[2], data.shape[3])

In [6]:
# Aplicação da segmentação (dom. tempo e frequência)

step = 29 # qual passo a cada deslizamento
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = signal.stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')


 (56, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (56, 2, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (56, 2, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Tarefa 2*: Separar os participantes e armazenar os dados em disco para execução das proximas tarefas


In [7]:
print(chunks_freq[0].shape, chunks_freq[1].shape)
print(chunks_time[0].shape, chunks_time[1].shape)

gabi_freq = chunks_freq[0]
gabi_time = chunks_time[0]

jesse_freq = chunks_freq[1]
jesse_time = chunks_time[1]

np.save("./datasets/gabi_freq", gabi_freq)
np.save("./datasets/gabi_time", gabi_time)

np.save("./datasets/jesse_freq", jesse_freq)
np.save("./datasets/jesse_time", jesse_time)

(2, 33, 33) (2, 33, 33)
(2, 33, 64) (2, 33, 64)
